# Informe de desempeño de PROESUR años 2015 - 2024

Este informe consta del desempeño del campus PROESUR desde el año 2015 hasta el año 2024. 

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
ALIASES = {
    "Cod_Estable" : ["Cod_Estable", "cod_estable"],
    "Cantidad_Estud" : ["Cantidad_Estud"],
    "Edad" : ["Edad", "EDAD"],
    "Logro_Mate" : ["Logro_Mate"],
    "Logro_Lect" : ["Logro_Lect"],
    "Desemp_Mate" : ["Desempeño_Mate"],
    "Desemp_Lect" : ["Desempeño_Lect"],
    "Measure_Mate" : ["Measure_Mate", "measure_mate"],
    "Measure_Lect" : ["Measure_Lect", "measure_lect"],
    "Service_Internet" : ["Sc_Servicio_Internet"],
    "Service_Electricity" : ["Sc_Electricidad"],
    "Rep_Grad" : ["Ed_Reptio_Algun_Grado"],
    "Ed_Trabaja_Actualmente" : ["Ed_Trabaja_Actualmente"],
    "Fm_Asistio_Escuela_Papa" : ["Fm_Asistio_Escuela_Papa"],
    "Fm_Asistio_Escuela_Mama" : ["Fm_Asistio_Escuela_Mama"],
    "Fm_Grado_Alcanzado_Papa" : ["Fm_Grado_Alcanzado_Papa"],
    "Fm_Grado_Alcanzado_Mama" : ["Fm_Grado_Alcanzado_Mama"],

}

In [13]:
def load_small_chunks(path: Path, code: str = "05-02-0232-46") -> pd.DataFrame:
    '''
    Extracts only the data we want from each xlsx file 
    ''' 
    df = pd.read_excel(path, engine="openpyxl")
    df = df[df["Cod_Estable"] == code]
    return df

def concat_dataframes(files: list[Path]) -> pd.DataFrame | None:
    df_list: list[pd.DataFrame] = []
    for f in files:
        df_list.append(load_small_chunks(f))
    try:
        result = pd.concat(df_list, ignore_index=True)
        return result
    except Exception as e:
        print(f"Error: {e}")
        return None
 

In [14]:
files = sorted(Path("data").glob("*.xlsx"))
result: pd.DataFrame | None = concat_dataframes(files)
if result is not None:
    print(result.shape, result.head())
else:
    print("Error")


(166, 385)      Cod_Estable  Cod_Evaluacion  Cantidad_Estud  Num_Boleta Indicadores  \
0  05-02-0232-46       503695901              42    201720.0         NaN   
1  05-02-0232-46       644817929              42    201689.0         NaN   
2  05-02-0232-46       899558295              42    201697.0         NaN   
3  05-02-0232-46       978064479              42    201687.0         NaN   
4  05-02-0232-46      1332275421              42    201699.0         NaN   

   Cod_Indicadores      Barcode Corr_Año              Region  Cod_Region  ...  \
0              NaN  50503695901       22  Región 5 o Central         5.0  ...   
1              NaN  50644817929       22  Región 5 o Central         5.0  ...   
2              NaN  50899558295       22  Región 5 o Central         5.0  ...   
3              NaN  50978064479       22  Región 5 o Central         5.0  ...   
4              NaN  51332275421       22  Región 5 o Central         5.0  ...   

  Gen_63_Modulo_O_Guia  Gen_63_Plataforma Gen

In [32]:


folder = Path("data")
files = list(folder.glob("*.xlsx"))

schemas = {}
for f in files:
    df = pd.read_excel(f, nrows=0)  # only headers
    schemas[f.name] = df.columns.tolist()

# Find total set and intersection
all_cols = set().union(*schemas.values())
common_cols = set.intersection(*map(set, schemas.values()))

print(f"Total unique columns: {len(all_cols)}")
print(f"Common columns: {len(common_cols)}")


Total unique columns: 623
Common columns: 0


In [24]:
result.head(3)

,Cod_Estable,Cod_Evaluacion,Cantidad_Estud,Num_Boleta,Indicadores,Cod_Indicadores,Barcode,Corr_Año,Region,Cod_Region,...,Gen_63_Modulo_O_Guia,Gen_63_Plataforma,Gen_63_Material_Periodicos,Gen_63_Otro_Recurso_Utilizó,Gen_63_Otro_Describa,Edad_RECO,Sexo_RECO,Cod_Sector,Sc_Neflix,Informacion_Prueba
0,05-02-0232-46,503695901,42,201720.0,NaN,NaN,50503695901,22,Región 5 o Central,5.0,...,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,05-02-0232-46,644817929,42,201689.0,NaN,NaN,50644817929,22,Región 5 o Central,5.0,...,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,05-02-0232-46,899558295,42,201697.0,NaN,NaN,50899558295,22,Región 5 o Central,5.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
result.tail(3)

,Cod_Estable,Cod_Evaluacion,Cantidad_Estud,Num_Boleta,Indicadores,Cod_Indicadores,Barcode,Corr_Año,Region,Cod_Region,...,Gen_63_Modulo_O_Guia,Gen_63_Plataforma,Gen_63_Material_Periodicos,Gen_63_Otro_Recurso_Utilizó,Gen_63_Otro_Describa,Edad_RECO,Sexo_RECO,Cod_Sector,Sc_Neflix,Informacion_Prueba
163,05-02-0232-46,9357138014,70,307113.0,NaN,NaN,59357138014,2024.0,Región 5 o Central,5.0,...,NaN,NaN,NaN,NaN,NaN,2.0,1.0,22.0,1.0,1.0
164,05-02-0232-46,9814811171,70,307082.0,NaN,NaN,59814811171,2024.0,Región 5 o Central,5.0,...,NaN,NaN,NaN,NaN,NaN,2.0,1.0,22.0,1.0,1.0
165,05-02-0232-46,9882787927,70,307108.0,NaN,NaN,59882787927,2024.0,Región 5 o Central,5.0,...,NaN,NaN,NaN,NaN,NaN,2.0,1.0,22.0,1.0,1.0


In [30]:
result["Cantidad_Estud"].head(10)

0    42
1    42
2    42
3    42
4    42
5    42
6    42
7    42
8    42
9    42
Name: Cantidad_Estud, dtype: int64